In [1]:
%pip install selenium
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\komal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\komal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver # type: ignore
from selenium.webdriver.common.by import By # type: ignore
from selenium.webdriver.chrome.service import Service # type: ignore
from selenium.webdriver.chrome.options import Options # type: ignore
from selenium.webdriver.common.action_chains import ActionChains # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.webdriver.support import expected_conditions as EC # type: ignore
import pandas as pd # type: ignore
from datetime import datetime
import os
import time

# Setup Selenium with ChromeDriver
options = Options()
options.add_argument("--headless")  # Run headlessly, without opening a window
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
service = Service(r'C:\Users\komal\Downloads\chromedriver.exe')  

In [3]:
# Initialize the WebDriver
# This line initializes a Chrome WebDriver instance, 
# which is used to automate interactions with the Chrome browser. 
# service and options are arguments that configure the WebDriver's 
# behavior, such as setting the path to the ChromeDriver executable 
# and specifying browser options.
driver = webdriver.Chrome(service=service, options=options)

# URL of the Amazon reviews page
def scrapreviews(url):
    driver.get(url)

    # Allow time for the page to load
    time.sleep(3)

    # Scroll to load more reviews (optional, depends on how many reviews are visible)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    # Scrape review data
    flag = 0
    reviews = []
    #Finds all elements on the page that match the XPath //div[@data-hook='review'], 
    #which targets review blocks containing individual reviews.
    review_blocks = driver.find_elements(By.XPATH, "//div[@data-hook='review']")
    #Iterates over each review block. Sets the flag to 1 to indicate that at least one review has 
    #been processed. Initializes an empty dictionary (review) to store data for the current review.
    for review_block in review_blocks:
        flag = 1
        review = {}
        review['Website'] = url
        review['Brand'] = "Life Extension"
        review['Dose'] = "1capsule"
        review['Mg'] = "125"
        review['Form'] = "Capsule"
        try:
            review['Title'] = review_block.find_element(By.XPATH, ".//a[@data-hook='review-title']").text.strip()
            rating_element = review_block.find_element(By.XPATH, ".//i[@data-hook='review-star-rating']/span[@class='a-icon-alt']")
            review['Rating'] = rating_element.get_attribute("innerHTML").strip()
            review['Rating'] = review['Rating'].split(' ',1)[0]
            review['Date'] = review_block.find_element(By.XPATH, ".//span[@data-hook='review-date']").text.strip()
            review['Date'] = review['Date'].split('on ', 1)[-1]
            month = review['Date'].split(' ', 1)[0]
            year = review['Date'].split(', ', 1)[-1] 
            review['Date'] = month + " " +year
            date_obj = datetime.strptime(review['Date'], "%B %Y")
            review['Date'] = date_obj.strftime("%m-%Y")
            review['Comment'] = review_block.find_element(By.XPATH, ".//span[@data-hook='review-body']").text.strip()
        except Exception as e:
            print(f"Error extracting review: {e}")
            continue
        reviews.append(review)
    if flag == 1:
        # Convert the data into a DataFrame
        df = pd.DataFrame(reviews)
        file_path = "test.csv"
        # Check if the file already exists
        if os.path.exists(file_path):
            # Append data to the existing file without writing the header
            df.to_csv(file_path, mode='a', index=False, header=False)
        else:
            # Write data to a new file with the header
            df.to_csv(file_path, mode='w', index=False, header=True)
        print("Scraping completed. Reviews saved to test.csv")

    # Close the WebDriver
    driver.quit()

In [4]:
#scrapreviews("https://www.amazon.com/Futurebiotics-Ashwagandha-Strength-Support-BioPerine/product-reviews/B0CQMHSPDV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews")

First in
